# Boosted Decision Tree Pipeline

In [1]:
import pandas as pd 

dataframe = pd.read_csv("SBAnational.csv", low_memory=False)
dataframe.head(3)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier


# Séparation des caractéristiques (X) et de la cible (y)
X = dataframe.drop('MIS_STAUS', axis=1)
y = dataframe['MIS_STAUS']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définition des colonnes numériques et catégorielles
colonnes_numeriques = X.select_dtypes(include=['int64', 'float64']).columns
colonnes_categorielles = X.select_dtypes(include=['object']).columns

# Création des étapes de prétraitement pour les colonnes numériques et catégorielles
preprocessing = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), colonnes_numeriques),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='inconnu')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), colonnes_categorielles)
    ]
)

# Création du pipeline complet
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Évaluation du modèle
score = pipeline.score(X_test, y_test)
print(f'Précision du modèle : {score:.2f}')

KeyError: "['MIS_STAUS'] not found in axis"